In [59]:
import sys
sys.path.append("/storage/BonnieH/selection_project/helper_functions")
from ABC_functions import *

In [60]:
# Code to write example loci to file:

def WriteExampleLoci(inFile, outFile):
    inFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/merged_filtered.txt'
    outFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/example_loci.txt'

    allele_freqs_file = open(inFile, 'r')
    results = open(outFile, 'w')
    
    codis_list = [('chr2', 1489651), ('chr5', 123775553), ('chr5', 150076322), ('chr7', 84160225), ('chr8', 124894863), \
                  ('chr11', 2171086), ('chr13', 82148025), ('chr16', 86352702), ('chr18', 63281667)]
        
    disease_list = [('chr19', 45770205), ('chr19', 13207859), ('chr13', 70139384), ('chr5', 146878729)]
    
    early_onset_list = [('chr6', 45315923), ('chr6', 45362057), ('chr6', 45474374), ('chr6', 45478639), \
                       ('chr6', 45506657), ('chr6', 45515702), ('chr6', 45523256), ('chr2', 176089200), \
                       ('chr2', 176094386)]
                  
    codisFile = '/storage/BonnieH/selection_project/annotations/hg38_codis.bed'
    diseaseFile = '/storage/BonnieH/selection_project/annotations/hg38_disease.txt'
    earlyonsetFile = '/storage/BonnieH/selection_project/annotations/hg38_earlyonset.txt'

    codis = open(codisFile, 'r')
    codis_dic = {}
    for line in codis:
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        name = info[5]
        codis_dic[(chrom, start)] = name
    codis.close()
        
    disease = open(diseaseFile, 'r')
    disease_dic = {}
    for line in disease:
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        name = info[5]
        disease_dic[(chrom, start)] = name
    disease.close()

    early = open(earlyonsetFile, 'r')
    early_dic = {}
    for line in early:
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        name = info[4]
        early_dic[(chrom, start)] = name
    early.close()

    Lines = allele_freqs_file.readlines()
    
    for line in Lines:
        
        # Get information from line
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        per = int(info[3])
        freqs = info[4]
            
        # Get info about reference allele (length in base pairs, repeat units)
        ref_length_bp = end - start + 1
        ref_length_ru = int(ref_length_bp/per)
            
        tup = (chrom, start)
            
            
        if (tup in disease_dic) or (tup in early_dic) or (tup in codis_dic):
            print(tup)
            results.write(line)
            '''
            elif per == 2 and ref_length_ru >= 11:
                results.write(line)
                
            elif per == 3 and ref_length_ru >= 5:
                results.write(line)
                
            elif per == 4 and ref_length_ru >= 7:
                results.write(line)
            '''
    results.close()        
    allele_freqs_file.close()
    print("Done")

In [61]:
# Code to filter out short loci based on optimal allele length threshold 

def FilterShortLoci(inFile, outFile):
    
    allele_freqs_file = open(inFile, 'r')
    results = open(outFile, 'w')
    
    num_lines_before_filt = 0
    num_lines_after_filt = 0
    
    for line in allele_freqs_file:
        num_lines_before_filt = num_lines_before_filt + 1
        
        # Get information from line
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        freqs = info[3]
        per = int(info[4])
        if freqs == '.':
            print(line)
        
        #  Only continue if freqs != '.'
        if freqs != '.' : 
            
            # Get info about reference allele (length in base pairs, repeat units)
            ref_length_bp = end - start + 1
            ref_length_ru = int(ref_length_bp/per)
            
            optimal_ru = Process_Freqs(freqs, per, end, start, False)
            
            if per == 4 and optimal_ru == 5:
                print(line)
            if per == 1 and optimal_ru >= 20:
                results.write(line)
                num_lines_after_filt = num_lines_after_filt + 1
                
            elif per == 2 and optimal_ru >= 11:
                results.write(line)
                num_lines_after_filt = num_lines_after_filt + 1 
                
            elif per == 3 and optimal_ru >= 5:
                results.write(line)
                num_lines_after_filt = num_lines_after_filt + 1
                
            elif per == 4 and optimal_ru >= 7:
                results.write(line)
                num_lines_after_filt = num_lines_after_filt + 1
                
            # Adding RUNX2 tracts?
            #elif chrom == "chr6" and start == 45442365:
             #   results.write(line)
                
            #elif chrom == "chr6" and start == 45457637:
             #   results.write(line)
            
    results.close()        
    allele_freqs_file.close()
    print("Done")
    print(num_lines_before_filt)
    print(num_lines_after_filt)

In [62]:
def main():
    inFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/allele_freqs_per_1234.txt'
    outFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/allele_freqs_filt_per_1234.txt'
    FilterShortLoci(inFile, outFile)

In [63]:
%%time
if __name__ == '__main__':
    main()

1	260054	260073	-4:2,0:6322,16:1,4:17	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	AP006222.1

1	349459	349478	0:4555,12:1,16:1,4:2776,8:195	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	RPL23AP24

1	786985	787004	-4:309,0:7221	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	AL669831.5

1	1694294	1694313	-4:1,0:7419,4:6	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	SLC35E2B

1	1846569	1846588	-4:1,-8:3,0:7509,4:14,8:1	4	AATC	0.0	1.0	0.0	0.0	0.0	0.0	GNB1

1	1941116	1941135	-12:37,-8:1,0:7489,4:1	4	AAAG	0.0	0.0	0.0	0.0	0.0	1.0	CFAP74

1	1949126	1949145	-4:16,0:4460,12:12,16:14,20:11,24:5,28:2,4:1197,60:1	4	AGGG	0.0	0.0	0.0	0.0	0.0	1.0	CFAP74

1	2099322	2099341	0:7530	4	AATG	0.0	0.0	0.0	0.0	0.0	1.0	PRKCZ

1	2101520	2101539	0:7527,4:1	4	AATG	0.0	0.0	0.0	0.0	0.0	1.0	PRKCZ

1	2243957	2243976	-4:1,0:6943,12:5,4:8,8:569	4	AAAG	0.0	0.0	0.0	0.0	0.0	1.0	SKI

1	2467181	2467200	-12:1,-8:1,0:7503,4:5,8:6	4	AGGG	0.0	0.0	0.0	0.0	0.0	1.0	PLCH2

1	2562206	2562225	0:7515,4:15	4	AATG	0.0	0.0	0.0	0.0	0.0	1.0	TNFRSF14

1	2578814	2578833	0:7528	4	AAAC	0.0	0.0	0.0	0.0	

1	15788592	15788611	0:7527,16:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	RPL12P14

1	15806257	15806276	0:7517,4:13	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	UQCRHL

1	15827865	15827884	0:5131,12:2,8:587	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	AL450998.1

1	15837968	15837987	0:7529,4:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	SPEN

1	15913451	15913470	0:7524,4:4	4	AATC	0.0	0.0	0.0	0.0	0.0	1.0	AL034555.1

1	15918142	15918161	0:7530	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	AL034555.1

1	15955157	15955176	0:7516,4:10	4	AAAC	0.0	1.0	0.0	0.0	0.0	0.0	ZBTB17

1	16207336	16207359	-4:5610,0:1920	4	AATG	0.0	0.0	0.0	0.0	0.0	1.0	ARHGEF19

1	16237674	16237693	0:7530	4	AATG	0.0	0.0	0.0	0.0	0.0	1.0	CPLANE2

1	16317413	16317432	-8:17,0:4915,12:36,16:2,4:13,8:2539	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	FBXO42

1	16329586	16329605	-4:1,0:7528,4:1	4	AATC	0.0	0.0	0.0	0.0	0.0	1.0	FBXO42

1	16360476	16360495	-4:4,-8:82,0:7409,4:35	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	SZRD1

1	16412783	16412802	-4:2,-8:2,0:7521,4:5	4	AAAG	0.0	1.0	0.0	0.0	0.0	0.0	SPATA21

1	16426255	164262


1	28487042	28487061	-4:1,-8:60,0:7467,4:2	4	ACAT	0.0	0.0	0.0	0.0	0.0	1.0	PHACTR4

1	28526294	28526313	-4:22,0:7503,12:1	4	AAAC	0.0	1.0	0.0	0.0	0.0	0.0	PRDX3P2

1	28549345	28549364	0:7525,4:3	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	TRNAU1AP

1	28551028	28551047	0:7525,4:3	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	TRNAU1AP

1	28608759	28608778	0:7527,4:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	TAF12

1	28615107	28615126	0:7524,4:2	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	TAF12

1	28616738	28616757	-4:1,0:7523,4:2	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	TAF12

1	28670154	28670173	-4:1,0:4303,12:2288,16:733,20:135,24:42,28:4,4:17,8:5	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	GMEB1

1	28674391	28674410	-4:2,0:7527,4:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	GMEB1

1	28676731	28676750	-4:495,-8:1,0:6001,4:5	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	GMEB1

1	28684859	28684878	0:7530	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	GMEB1

1	28767501	28767520	-4:1,0:7507,4:22	4	AAAT	0.0	1.0	0.0	0.0	0.0	0.0	YTHDF2

1	28909814	28909833	0:4710,4:2818	4	AGAT	0.0	1.0	0.0	0.0	0.0	0.0	EPB41

1	2


1	44480408	44480427	-4:2,0:7519,4:3	4	AATT	0.0	0.0	0.0	0.0	0.0	1.0	MIR5584

1	44565288	44565307	0:7503,12:3	4	AGGG	0.0	0.0	0.0	0.0	0.0	1.0	MIR5584

1	44692732	44692751	-4:1,0:7526,4:3	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	ARMH1

1	44742104	44742123	-4:6,0:7513,12:1	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	KIF2C

1	44766058	44766077	0:7529,4:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	AL592166.1

1	44801785	44801804	-4:2,0:7422	4	AGGG	0.0	0.0	0.0	0.0	0.0	1.0	PLK3

1	44841168	44841187	-4:2,0:7396,4:100	4	AAAC	0.0	1.0	0.0	0.0	0.0	0.0	PTCH2

1	44932427	44932446	-4:1,0:7510,4:17	4	AAAC	0.0	1.0	0.0	0.0	0.0	0.0	RNA5SP47

1	45102853	45102872	0:7530	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	OSTCP5

1	45192507	45192526	0:7472,4:56	4	AAGG	0.0	0.0	0.0	0.0	0.0	1.0	ZSWIM5

1	45222636	45222655	-4:37,0:7481,4:12	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	ZSWIM5

1	45277119	45277138	0:7526,12:1,4:1	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	LINC01144

1	45294983	45295002	0:7523,4:1	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	LINC01144

1	45313841	45313860	0:7530	4	AAAT	0.0	0.

1	60928437	60928456	0:7527,4:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	NFIA-AS2

1	61048243	61048262	-4:1,0:7504,4:25	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	NFIA

1	61077360	61077379	-4:4,0:7470,4:44	4	AGCG	0.0	0.0	1.0	0.0	0.0	0.0	NFIA

1	61459266	61459285	0:7242,12:8,4:277,8:1	4	AAAC	0.0	0.0	0.0	1.0	0.0	0.0	AC099791.3

1	61741051	61741070	-4:1,0:7529	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	PATJ

1	61745272	61745291	0:7529,4:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	PATJ

1	61820338	61820357	0:7528,4:2	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	RNU6-414P

1	61835391	61835410	-4:1,0:7497	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	RNU6-1177P

1	61925200	61925268	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	PATJ

1	62008751	62008770	0:7523,20:1,4:4	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	LAMTOR5P1

1	62020815	62020834	0:7530	4	AATG	0.0	0.0	0.0	0.0	0.0	1.0	LAMTOR5P1

1	62078270	62078289	0:7526,4:4	4	ACAT	0.0	0.0	0.0	0.0	0.0	1.0	MIR3116-1

1	62118624	62118643	0:7429,4:100,8:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	PATJ

1	62173970	62173989	-4:335,-8:4,0:7182,4:1	4	AAAC	0.0	0.0	0.

...


21	8661540	8661559	0:5802,4:481,8:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	CR381572.2

21	8988058	8988077	.	4	ACGG	0.0	0.0	0.0	0.0	0.0	1.0	CR392039.1

21	9014419	9014438	-4:1,0:47	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	CR392039.4

21	9161490	9161509	-8:19,0:6165,12:406,16:56,20:5,4:4,8:431	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	TEKT4P2

21	9683213	9683232	-4:100,-8:1,0:6545,4:856,8:4	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	RF00003

21	9707956	9707975	-4:4,0:7429,4:73	4	ACAG	0.0	0.0	0.0	0.0	0.0	1.0	RF00003

21	10149470	10149489	0:5590,4:1890,8:2	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	CR382287.1

21	10382855	10382874	0:259,4:15	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	AF254983.2

21	10383250	10383269	0:314	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	AF254983.2

21	10418381	10418428	-28:1,-4:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	BAGE2

21	13173464	13173483	-4:1,0:7217,4:270	4	AACC	0.0	0.0	0.0	0.0	0.0	1.0	AJ239321.1

21	13215718	13215737	0:7493,4:1	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	AJ239318.1

21	13335024	13335043	-4:141,-8:1,0:4617,4:2745	4	AAAG	0.0	0.0	0.0	


21	30530661	30530680	0:7502	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	AP000567.1

21	30607991	30608010	-4:2,0:7490,4:14	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	KRTAP6-1

21	30851955	30851974	0:7504,4:2	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	KRTAP7-1

21	30855649	30855668	-8:1,0:7457,4:48	4	ACAT	0.0	0.0	0.0	0.0	0.0	1.0	KRTAP11-1

21	31369232	31369251	-4:459,0:3722,12:561,16:5,4:12,8:1	4	AAAG	0.0	0.0	0.0	0.0	0.0	1.0	AP000248.1

21	31428995	31429014	-4:1,0:7432,12:44,16:4,4:25	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	AP000563.1

21	31459115	31459134	0:5361,4:2144,8:1	4	AATC	0.0	0.0	0.0	0.0	0.0	1.0	AP000563.1

21	31512794	31512813	-4:3,0:6117,4:1382	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	TIAM1

21	31526702	31526721	0:7506	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	TIAM1

21	31565278	31565297	0:7505,4:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	TIAM1

21	31631701	31631720	0:26	4	AATT	0.0	0.0	0.0	0.0	0.0	1.0	FBXW11P1

21	31642620	31642639	0:7503,20:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	FBXW11P1

21	31736429	31736448	-4:10,0:7495,4:1	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0

22	10638667	10638686	0:5978,4:336,8:2	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	RF00004

22	11035817	11035836	0:6	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	CU104787.1

22	11036206	11036225	0:2	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	CU104787.1

22	11255753	11255772	0:6423,12:216,16:26,20:1,24:1,4:1	4	AGGC	0.0	0.0	0.0	0.0	0.0	1.0	RF00002

22	11353376	11353395	-12:3,-4:41,0:1406	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	RF00002

22	11618170	11618189	-4:2,0:6978,4:8	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	AC138776.1

22	11929144	11929163	-4:606,0:1771,4:317	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	AC138776.1

22	12416140	12416159	-4:9,0:6995,4:2	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	FRG1GP

22	15241090	15241133	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	RF00026

22	16433027	16433046	-4:4,0:7234,12:69,16:29,20:24,24:1,4:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	PABPC1P9

22	16586489	16586508	0:7496,4:4	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	KCNMB3P1

22	16588459	16588478	-4:1,0:7476,4:17	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	CCT8L2

22	16589008	16589027	0:7498,4:2	4	AAAT	0.0	0.0	0.0	0.0	0.0	1


22	29078260	29078279	-4:5,0:7496,4:1	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	KREMEN1

22	29090973	29090988	0:3656,4:3846	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	KREMEN1

22	29355461	29355480	-8:1,0:7492,4:9	4	AAAC	0.0	1.0	0.0	0.0	0.0	0.0	AP1B1

22	29466986	29467005	-4:136,-8:1,0:7347,4:4	4	AAGG	0.0	0.0	0.0	0.0	0.0	1.0	RFPL1S

22	29477312	29477331	0:7497,4:5	4	ATCC	0.0	0.0	0.0	0.0	0.0	1.0	RFPL1S

22	29502789	29502808	0:7501,4:1	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	THOC5

22	29527423	29527442	-4:1,0:7452,4:49	4	AAAC	0.0	1.0	0.0	0.0	0.0	0.0	AC005529.1

22	29611519	29611538	-4:4,0:7487,4:11	4	AAAC	0.0	1.0	0.0	0.0	0.0	0.0	RPEP4

22	29839433	29839452	0:7499,4:3	4	AAAT	0.0	0.0	0.0	0.0	1.0	1.0	ASCC2

22	29957416	29957459	-12:133,-16:56,-20:5,-24:2768,-4:632,-8:1902,0:1689,12:1,4:307,8:9	4	AAAT	0.0	1.0	0.0	0.0	0.0	0.0	MTMR3

22	29964330	29964349	-4:5,0:7480,12:1,4:12	4	AAAC	0.0	1.0	0.0	0.0	0.0	0.0	MTMR3

22	29980769	29980788	0:7490,4:12	4	AAAG	0.0	1.0	0.0	0.0	0.0	0.0	MTMR3

22	29985501	29985520	0:7482,4:18	4	AAAG	0.0

22	41647265	41647284	0:7470,4:32	4	AAAT	0.0	1.0	0.0	0.0	0.0	0.0	RF00019

22	41692403	41692426	-4:5664,0:1830	4	AAAG	0.0	1.0	0.0	0.0	0.0	0.0	C22orf46

22	42116072	42116091	-4:1,0:7457,12:3,16:3,4:2	4	AAAG	0.0	0.0	0.0	0.0	0.0	1.0	NDUFA6-DT

22	42364535	42364554	-4:2,0:7500	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	LINC01315

22	42420768	42420787	-4:372,0:7126,4:2	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	NFAM1

22	42451561	42451580	-4:17,0:7478,4:1	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	AL022316.1

22	42488240	42488259	0:7498,12:1,4:3	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	SERHL

22	42658136	42658155	0:7438,12:5,16:2,8:3	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	CYB5R3

22	42673510	42673529	-4:13,0:7487,4:2	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	A4GALT

22	42730625	42730644	0:7448,4:54	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	A4GALT

22	42777847	42777866	0:7496	4	AAAT	0.0	0.0	0.0	0.0	0.0	1.0	RPL5P34

22	42855860	42855879	-12:1,-4:3230,0:4248,4:3	4	AATT	0.0	0.0	0.0	0.0	0.0	1.0	ARFGAP3

22	42956494	42956513	0:7496,4:6	4	AATG	0.0	0.0	0.0	0.0	0.0	1.0	PACSIN